### Load QM9 model and predict

In [1]:
from megnet.models import MEGNetModel
import numpy as np
from operator import itemgetter
import json

def get_graph_from_doc(doc):
    """
    Convert a json document into a megnet graph
    """
    atom = [i['atomic_num'] for i in doc['atoms']]

    index1_temp = [i['a_idx'] for i in doc['atom_pairs']]
    index2_temp = [i['b_idx'] for i in doc['atom_pairs']]
    bond_temp = [i['spatial_distance'] for i in doc['atom_pairs']]

    index1 = index1_temp + index2_temp
    index2 = index2_temp + index1_temp
    bond = bond_temp + bond_temp
    sort_key = np.argsort(index1)
    it = itemgetter(*sort_key)

    index1 = it(index1)
    index2 = it(index2)
    bond = it(bond)
    graph = {'atom': atom, 'bond': bond, 'index1': index1, 'index2': index2, 'state': [[0, 0]]}
    return graph
    
# load an example qm9 document
with open('../megnet/data/tests/qm9/000001.json', 'r') as f:
    doc = json.load(f)
# convert to a graph
graph = get_graph_from_doc(doc)

In [2]:
graph

{'atom': [6, 1, 1, 1, 1],
 'bond': (1.0919181,
  1.0919425,
  1.0918945,
  1.0919342,
  1.7830887,
  1.783101,
  1.7831048,
  1.0919181,
  1.7831084,
  1.7831007,
  1.0919425,
  1.7830887,
  1.7831084,
  1.783101,
  1.7831069,
  1.0918945,
  1.7831048,
  1.7831007,
  1.0919342,
  1.7831069),
 'index1': (0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4),
 'index2': (1, 2, 3, 4, 2, 3, 4, 0, 3, 4, 0, 1, 2, 1, 4, 0, 1, 2, 0, 3),
 'state': [[0, 0]]}

In [3]:
# all target names
names = ['mu', 'alpha', 'HOMO', 'LUMO', 'gap', 'R2', 'ZPVE', 'U0', 'U', 'H', 'G', 'Cv', 'omega1']


y_pred = []
y_true = []

print('*** Result Comparisons ***')
print('Target\tMEGNet\tQM9')

for i in names:
    model = MEGNetModel.from_file('../mvl_models/qm9-2018.6.1/' + i+'.hdf5')
    pred = model.predict_graph(graph)
    y_pred.append(pred)
    y_true.append(doc['mol_info'][i])
    print('%s\t%.3f\t%.3f' %(i, y_pred[-1], float(y_true[-1])))


*** Result Comparisons ***
Target	MEGNet	QM9


2024-01-22 19:42:11.832003: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 19:42:11.844875: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 19:42:11.844961: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-01-22 19:42:11.845497: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

mu	-0.008	0.000
alpha	13.127	13.210
HOMO	-10.557	-10.550
LUMO	3.241	3.186
gap	13.622	13.736
R2	35.974	35.364
ZPVE	1.215	1.218
U0	-17.166	-17.172
U	-17.353	-17.286
H	-17.420	-17.389
G	-16.107	-16.152
Cv	6.427	6.469
omega1	3151.625	3151.708


### Predict from SMILES

In [4]:
from megnet.utils.molecule import get_pmg_mol_from_smiles

MODEL_NAME = 'HOMO'

model = MEGNetModel.from_file('../mvl_models/qm9-2018.6.1/%s.hdf5' % MODEL_NAME)

In [5]:
# The smiles of qm9:000001 is just C
mol1 = get_pmg_mol_from_smiles('C')
model.predict_structure(mol1)

array([-10.557697], dtype=float32)

In [6]:

model.predict_graph(graph)

array([-10.556635], dtype=float32)

The result matches with previous results when we compute it from pre-computed graph